## Homework


### Dataset

In this homework, we'll build a model for predicting if we have an image of a dino or a dragon. For this, we will use the "Dino or Dragon?" dataset that can be downloaded from [Kaggle](https://www.kaggle.com/datasets/agrigorev/dino-or-dragon). 

You can get a wget-able version [here](https://github.com/alexeygrigorev/dino-or-dragon/releases/download/data/dino-dragon.zip):

```bash
wget https://github.com/alexeygrigorev/dino-or-dragon/releases/download/data/dino-dragon.zip
unzip dino-dragon.zip
```

In the lectures we saw how to use a pre-trained neural network. In the homework, we'll train a much smaller model from scratch. 

**Note:** You will need an environment with a GPU for this homework. We recommend to use [Saturn Cloud](https://bit.ly/saturn-mlzoomcamp). 
You can also use a computer without a GPU (e.g. your laptop), but it will be slower.

In [ ]:
# download and unzip dataset
!wget https://github.com/alexeygrigorev/dino-or-dragon/releases/download/data/dino-dragon.zip
!unzip dino-dragon.zip

### Data Preparation

The dataset contains around 1900 images of dinos and around 1900 images of dragons. 

The dataset contains separate folders for training and validation.

In [2]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib

Using matplotlib backend: agg


In [3]:
from tensorflow import keras

### Model

For this homework we will use Convolutional Neural Network (CNN). Like in the lectures, we'll use Keras.

You need to develop the model with following structure:

* The shape for input should be `(150, 150, 3)`
* Next, create a convolutional layer ([`Conv2D`](https://keras.io/api/layers/convolution_layers/convolution2d/)):
    * Use 32 filters
    * Kernel size should be `(3, 3)` (that's the size of the filter)
    * Use `'relu'` as activation 
* Reduce the size of the feature map with max pooling ([`MaxPooling2D`](https://keras.io/api/layers/pooling_layers/max_pooling2d/))
    * Set the pooling size to `(2, 2)`
* Turn the multi-dimensional result into vectors using a [`Flatten`](https://keras.io/api/layers/reshaping_layers/flatten/) layer
* Next, add a `Dense` layer with 64 neurons and `'relu'` activation
* Finally, create the `Dense` layer with 1 neuron - this will be the output
    * The output layer should have an activation - use the appropriate activation for the binary classification case

As optimizer use [`SGD`](https://keras.io/api/optimizers/sgd/) with the following parameters:

* `SGD(lr=0.002, momentum=0.8)`

For clarification about kernel size and max pooling, check [Office Hours](https://www.youtube.com/watch?v=1WRgdBTUaAc).

In [4]:
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten

In [5]:
model = keras.Sequential([
    Input(shape=(150,150,3)),
    Conv2D(32, 3, activation='relu'),
    MaxPooling2D(2),
    Flatten(),
    Dense(64, 'relu'),
    Dense(1, 'sigmoid')
])

In [6]:
from keras.optimizers import SGD

In [7]:
sgd_optim = SGD(learning_rate=0.002, momentum=0.8)

In [8]:
model.compile(optimizer=sgd_optim,
              loss='binary_crossentropy',
              metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 175232)            0         
                                                                 
 dense (Dense)               (None, 64)                11214912  
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 11,215,873
Trainable params: 11,215,873
Non-trainable params: 0
____________________________________________

### Question 1

Since we have a binary classification problem, what is the best loss function for us?

- `binary crossentropy`
- `focal loss`
- `mean squared error`
- `categorical crossentropy`

Note: since we specify an activation for the output layer, we don't need to set `from_logits=True`


**Answer: binary crossentropy**

### Question 2

What's the total number of parameters of the model? You can use the `summary` method for that. 

- 9215873
- 11215873
- 14215873
- 19215873

**Answer: 11215873**

### Generators and Training

For the next two questions, use the following data generator for both train and validation:

```python
ImageDataGenerator(rescale=1./255)
```

* We don't need to do any additional pre-processing for the images.
* When reading the data from train/val directories, check the `class_mode` parameter. Which value should it be for a binary classification problem?
* Use `batch_size=20`
* Use `shuffle=True` for both training and validation 

For training use `.fit()` with the following params:

```python
model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)
```

In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [10]:
generator = ImageDataGenerator(rescale=1./255)
train_ds = generator.flow_from_directory('/content/train',
                                         batch_size=20,
                                         shuffle=False,
                                         target_size=(150,150),
                                         class_mode='binary')
val_ds = generator.flow_from_directory('/content/test',
                                         batch_size=20,
                                         shuffle=False,
                                         target_size=(150,150),
                                         class_mode='binary')

Found 1594 images belonging to 2 classes.
Found 394 images belonging to 2 classes.


In [11]:
history = model.fit(train_ds, epochs=10, validation_data=val_ds)

Epoch 1/10
80/80 [==============================] - 17s 119ms/step - loss: 0.7386 - accuracy: 0.4925 - val_loss: 0.6852 - val_accuracy: 0.5558
Epoch 2/10
80/80 [==============================] - 9s 116ms/step - loss: 0.6825 - accuracy: 0.5960 - val_loss: 0.6449 - val_accuracy: 0.6447
Epoch 3/10
80/80 [==============================] - 9s 115ms/step - loss: 0.6309 - accuracy: 0.6468 - val_loss: 0.5927 - val_accuracy: 0.8147
Epoch 4/10
80/80 [==============================] - 9s 115ms/step - loss: 0.6147 - accuracy: 0.6681 - val_loss: 0.5580 - val_accuracy: 0.7030
Epoch 5/10
80/80 [==============================] - 10s 123ms/step - loss: 0.5166 - accuracy: 0.7698 - val_loss: 0.4422 - val_accuracy: 0.8350
Epoch 6/10
80/80 [==============================] - 9s 116ms/step - loss: 0.4695 - accuracy: 0.7735 - val_loss: 0.4206 - val_accuracy: 0.8528
Epoch 7/10
80/80 [==============================] - 9s 112ms/step - loss: 0.4228 - accuracy: 0.8269 - val_loss: 0.4313 - val_accuracy: 0.7944
Epoc

### Question 3

What is the median of training accuracy for all the epochs for this model?

- 0.40
- 0.60
- 0.90
- 0.20


In [12]:
round(np.median(history.history['accuracy']),2)

0.77

**Answer: 0.9**

### Question 4

What is the standard deviation of training loss for all the epochs for this model?

- 0.11
- 0.66
- 0.99
- 0.33


In [13]:
round(np.std(history.history['loss']),2)

0.13

**Answer: 0.11**

### Data Augmentation

For the next two questions, we'll generate more data using data augmentations. 

Add the following augmentations to your training data generator:

* `rotation_range=40,`
* `width_shift_range=0.2,`
* `height_shift_range=0.2,`
* `shear_range=0.2,`
* `zoom_range=0.2,`
* `horizontal_flip=True,`
* `fill_mode='nearest'`

In [14]:
aug_generator = ImageDataGenerator( rescale=1./255,
                                    rotation_range=40,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    horizontal_flip=True,
                                    fill_mode='nearest'
)

In [15]:
train_ds = aug_generator.flow_from_directory('/content/train',
                                            batch_size=20,
                                            shuffle=True,
                                            target_size=(150,150),
                                            class_mode='binary')
val_ds = aug_generator.flow_from_directory('/content/test',
                                            batch_size=20,
                                            shuffle=True,
                                            target_size=(150,150),
                                            class_mode='binary')

Found 1594 images belonging to 2 classes.
Found 394 images belonging to 2 classes.


### Question 5 

Let's train our model for 10 more epochs using the same code as previously.
Make sure you don't re-create the model - we want to continue training the model
we already started training.

What is the mean of validation loss for all the epochs for the model trained with augmentations?

- 0.15
- 0.77
- 0.37
- 0.97

In [16]:
history = model.fit(train_ds, epochs=10, validation_data=val_ds)

Epoch 1/10
80/80 [==============================] - 18s 220ms/step - loss: 0.4269 - accuracy: 0.8061 - val_loss: 0.4471 - val_accuracy: 0.7716
Epoch 2/10
80/80 [==============================] - 18s 220ms/step - loss: 0.4236 - accuracy: 0.8055 - val_loss: 0.3872 - val_accuracy: 0.8503
Epoch 3/10
80/80 [==============================] - 19s 232ms/step - loss: 0.3987 - accuracy: 0.8243 - val_loss: 0.3718 - val_accuracy: 0.8401
Epoch 4/10
80/80 [==============================] - 18s 231ms/step - loss: 0.3854 - accuracy: 0.8319 - val_loss: 0.3986 - val_accuracy: 0.8173
Epoch 5/10
80/80 [==============================] - 17s 219ms/step - loss: 0.4055 - accuracy: 0.8237 - val_loss: 0.4222 - val_accuracy: 0.8096
Epoch 6/10
80/80 [==============================] - 18s 230ms/step - loss: 0.3917 - accuracy: 0.8206 - val_loss: 0.3895 - val_accuracy: 0.8350
Epoch 7/10
80/80 [==============================] - 17s 219ms/step - loss: 0.3959 - accuracy: 0.8256 - val_loss: 0.3591 - val_accuracy: 0.8426

In [17]:
round(np.mean(history.history['val_loss']),2)

0.38

**Answer: 0.37**

### Question 6

What's the average of validation accuracy for the last 5 epochs (from 6 to 10)
for the model trained with augmentations?

- 0.84
- 0.54
- 0.44
- 0.24

In [18]:
round(np.average(history.history['val_accuracy'][-5:]), 2)

0.84

**Answer: 0.84**